# Study Experiment Design Generation

This notebook analyzes the characteristics of different space-filling experiment design generation techniques for different input spaces.

In [ ]:
from typing import Optional, Annotated

import raxpy
import raxpy.spaces.root as s
import raxpy.does.projected_lhs as p_lhs

In [ ]:
def generate_designs():

    s.
    
    pass